# 📊 CRM Analytics - RFM Analizi

Bu notebook'ta FLO müşteri verileri üzerinde RFM analizi yapacağız.

**İçerik:**
1. Veri Yükleme ve Keşif
2. Veri Hazırlama
3. RFM Metriklerini Hesaplama
4. RFM Skorlama
5. Segmentasyon
6. Görselleştirme

## 1. Kütüphaneleri İçe Aktarma

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns

# Görselleştirme ayarları
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

# Pandas ayarları
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

print("✅ Kütüphaneler yüklendi!")

## 2. Veri Yükleme

In [ ]:
# Veri yükleme
df = pd.read_csv("../data/flo_data_20k.csv")

print(f"Veri boyutu: {df.shape}")
print(f"Satır: {df.shape[0]:,}, Sütun: {df.shape[1]}")

In [ ]:
# İlk 5 satır
df.head()

In [ ]:
# Veri tiplerine bakış
df.dtypes

In [ ]:
# İstatistiksel özet
df.describe().T

## 3. Veri Hazırlama

In [ ]:
# Tarih dönüşümleri
date_columns = [col for col in df.columns if 'date' in col]
for col in date_columns:
    df[col] = pd.to_datetime(df[col])

print("✅ Tarih dönüşümleri tamamlandı")

In [ ]:
# Omnichannel toplam değişkenler
df["order_num_total"] = df["order_num_total_ever_online"] + df["order_num_total_ever_offline"]
df["customer_value_total"] = df["customer_value_total_ever_online"] + df["customer_value_total_ever_offline"]

print("✅ Yeni değişkenler oluşturuldu")

## 4. RFM Metriklerini Hesaplama

In [ ]:
# Analiz tarihi
analysis_date = df["last_order_date"].max() + dt.timedelta(days=2)
print(f"Analiz tarihi: {analysis_date.date()}")

In [ ]:
# RFM hesaplama
rfm = df.groupby('master_id').agg({
    'last_order_date': lambda date: (analysis_date - date.max()).days,
    'order_num_total': lambda num: num.sum(),
    'customer_value_total': lambda value: value.sum()
})

rfm.columns = ['recency', 'frequency', 'monetary']

rfm.head()

In [ ]:
# RFM istatistikleri
rfm.describe().T

## 5. RFM Skorlama

In [ ]:
# Skorlama
rfm["recency_score"] = pd.qcut(rfm['recency'], 5, labels=[5, 4, 3, 2, 1])
rfm["frequency_score"] = pd.qcut(rfm['frequency'].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])
rfm["monetary_score"] = pd.qcut(rfm['monetary'], 5, labels=[1, 2, 3, 4, 5])

# RF Score
rfm["RF_SCORE"] = rfm['recency_score'].astype(str) + rfm['frequency_score'].astype(str)

rfm.head()

## 6. Segmentasyon

In [ ]:
# Segment haritası
seg_map = {
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at_risk',
    r'[1-2]5': 'cant_loose',
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loyalists',
    r'5[4-5]': 'champions'
}

rfm['segment'] = rfm['RF_SCORE'].replace(seg_map, regex=True)

print("✅ Segmentler oluşturuldu")

In [ ]:
# Segment dağılımı
rfm['segment'].value_counts()

## 7. Görselleştirme

In [ ]:
# Segment dağılımı pasta grafik
plt.figure(figsize=(10, 6))
rfm['segment'].value_counts().plot(kind='pie', autopct='%1.1f%%')
plt.title('RFM Segment Dağılımı')
plt.ylabel('')
plt.show()

In [ ]:
# Segment ortalamaları
segment_summary = rfm.groupby('segment').agg({
    'recency': 'mean',
    'frequency': 'mean',
    'monetary': 'mean'
}).round(2)

segment_summary

In [ ]:
# Segment ortalamaları bar plot
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

segment_summary['recency'].plot(kind='barh', ax=axes[0], color='skyblue')
axes[0].set_title('Ortalama Recency')
axes[0].set_xlabel('Gün')

segment_summary['frequency'].plot(kind='barh', ax=axes[1], color='lightgreen')
axes[1].set_title('Ortalama Frequency')
axes[1].set_xlabel('Alışveriş Sayısı')

segment_summary['monetary'].plot(kind='barh', ax=axes[2], color='salmon')
axes[2].set_title('Ortalama Monetary')
axes[2].set_xlabel('TL')

plt.tight_layout()
plt.show()

## 8. Sonuçları Kaydetme

In [ ]:
# CSV'ye kaydetme
rfm.to_csv('../outputs/rfm_segments_notebook.csv')
print("✅ Sonuçlar kaydedildi: outputs/rfm_segments_notebook.csv")

## 9. Özet

✅ RFM analizi tamamlandı!  
✅ Müşteriler 10 segmente ayrıldı  
✅ Segment bazlı istatistikler hesaplandı  
✅ Görselleştirmeler oluşturuldu  
✅ Sonuçlar kaydedildi  